In [12]:
from hello.hello3 import HelloApp
from hello import hello3
from hello import hello
from hello.pid.ag_pid import agpid
from hello.pid.ag_pid.agpid import *

In [13]:
def runagpid(r):
    try:
        r.doall()
    except Exception as e:
        import traceback
        traceback.print_exc()
    finally:
        return r
    
def test1(ip='192.168.1.21'):
    p = 0.1
    i = 0.01
    d = 0
    info=PIDInfo(p, i, d, beta=0)
    app = hello3.open_hello(ip)
    tests = []
    for sp in (8, 10, 12, 14, 30, 55):
        t = AgOffToAutoTest(info, 0, sp, app)
        tests.append(t)
    for start_pv in (10, 12, 14):
        t1 = AgManToAutoTest(info, start_pv, start_pv-2, app)
        t2 = AgAutoToAutoTest(info, start_pv, start_pv-2, app)
        tests.extend((t1, t2))
    for start_pv in (8, 10, 12):
        t1 = AgManToAutoTest(info, start_pv, start_pv+2, app)
        t2 = AgAutoToAutoTest(info, start_pv, start_pv+2, app)
        tests.extend((t1, t2))
    r = PIDRunner(tests, wb_name='3L Mag Drive Agitation Test.xlsx')
    return runagpid(r)

In [14]:
r = test1()

04/11/17 17:57:09 INFO <runall>: Running test 1 of 18: P:0.100 I:0.010 D:0.0000 SP:8
04/11/17 17:57:09 INFO <run>: Beginning test run for OffToAuto SP:8.0
04/11/17 17:57:09 INFO <setup_settings>: Setting PID Settings
04/11/17 17:57:13 INFO <setup_test>: Setting Agitation to OFF and waiting 30 seconds
04/11/17 17:57:13 INFO <off>: Setting Agitation to Off
04/11/17 17:57:43 INFO <start_test>: Starting Test
04/11/17 17:57:43 INFO <auto>: Setting Agitation to 8RPM
04/11/17 17:57:43 INFO <wait_for_settle>: Waiting for PV to settle
04/11/17 17:57:43 INFO <wait_for_settle>: Update: PV: 0.00 SP: 8.00 Settle Remaining: 120 Timeout: 179
04/11/17 17:57:58 INFO <wait_for_settle>: Update: PV: 9.18 SP: 8.00 Settle Remaining: 120 Timeout: 164
04/11/17 17:58:13 INFO <wait_for_settle>: Update: PV: 8.22 SP: 8.00 Settle Remaining: 115 Timeout: 149
04/11/17 17:58:28 INFO <wait_for_settle>: Update: PV: 7.74 SP: 8.00 Settle Remaining: 100 Timeout: 134
04/11/17 17:58:44 INFO <wait_for_settle>: Update: PV: 8.

In [15]:
from hello.hello3 import HelloApp, NotLoggedInError
from time import sleep
import time

def _bn():
    i = 1
    while True:
        yield "temp 1.%d"%i
        i += 1
bn = _bn().__next__

def call(f, *args):
    print("Calling ", f.__name__, repr(args), sep="")
    for i in range(100):
        try:
            return f(*args)
        except NotLoggedInError:
            h.login()
        except Exception as e:
            print(repr(e))
            

In [16]:
h = HelloApp('192.168.1.21')

In [17]:
hour = 3600
h.login()
if call(h.batchrunning):
    call(h.endbatch)

call(h.startbatch, bn())
call(h.settemp, 0, 37)
call(time.sleep, 8*hour)
call(h.endbatch)
call(h.startbatch, bn())
call(h.temp, 2, 37)

Calling batchrunning()
Calling startbatch('temp 1.1',)
Calling settemp(0, 37)
Calling sleep(28800,)
Calling endbatch()
Calling startbatch('temp 1.2',)
ServerCallError("Can't start a new batch - temp 1.1 batch is currently running.\nCustom Error Message.vi->Set Start Batch for Web UI.vi->Webservice Command Handler.vi->PBS_WebServer.vi->PBS_WebServer.vi.ProxyCaller 5041",)
ServerCallError("Can't start a new batch - temp 1.1 batch is currently running.\nCustom Error Message.vi->Set Start Batch for Web UI.vi->Webservice Command Handler.vi->PBS_WebServer.vi->PBS_WebServer.vi.ProxyCaller 5041",)
ServerCallError("Can't start a new batch - temp 1.1 batch is currently running.\nCustom Error Message.vi->Set Start Batch for Web UI.vi->Webservice Command Handler.vi->PBS_WebServer.vi->PBS_WebServer.vi.ProxyCaller 5041",)
ServerCallError("Can't start a new batch - temp 1.1 batch is currently running.\nCustom Error Message.vi->Set Start Batch for Web UI.vi->Webservice Command Handler.vi->PBS_WebServe

AttributeError: 'HelloApp' object has no attribute 'temp'

In [63]:
call(h.login)
call(h.endbatch)
call(h.startbatch, "real temp 1.24")
call(h.settemp, 2, 37)

Calling login()
Calling endbatch()
JSONDecodeError('Expecting value: line 1 column 1 (char 0)',)
Calling startbatch('real temp 1.24',)
Calling settemp(2, 37)


In [64]:
b

b'{\n"result": "False",\n"message": "No user associated with the setendbatch call Custom Error Message.vi->Determine User Permission for Call.vi->Webservice Command Handler.vi->PBS_WebServer.vi->PBS_WebServer.vi.ProxyCaller 5040"\n}'

In [65]:
b = h.send_request("?call=setstartbatch&val1=foo")

In [66]:
b.content

b'<?xml version="1.0" encoding="UTF-8" standalone="no" ?><Reply><Result>False</Result><Message>No user associated with the setstartbatch call\nCustom Error Message.vi-&gt;Determine User Permission for Call.vi-&gt;Webservice Command Handler.vi-&gt;PBS_WebServer.vi-&gt;PBS_WebServer.vi.ProxyCaller 5040</Message></Reply>'

In [70]:
from hello.hello3 import HelloXML
HelloXML(b.content).data

'No user associated with the setstartbatch call\nCustom Error Message.vi->Determine User Permission for Call.vi->Webservice Command Handler.vi->PBS_WebServer.vi->PBS_WebServer.vi.ProxyCaller 5040'